<a href="https://colab.research.google.com/github/KevinJayne2023/ComfyUI_Custom_Image_Generator/blob/main/Flux_Lora_ComfyUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Notes
- Uncheck `UPDATE_COMFY_UI`, and `USE_COMFYUI_MANAGER` and `INSTALL_CUSTOM_NODES` after first run if `USE_GOOGLE_DRIVE` is checked.


### Install Dependencies

In [ ]:
# #@title Environment Setup

from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_CUSTOM_NODES'] = INSTALL_CUSTOM_NODES
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f "ComfyUI-Manager/check.sh" ] && chmod 755 ComfyUI-Manager/check.sh
  ![ -f "ComfyUI-Manager/scan.sh" ] && chmod 755 ComfyUI-Manager/scan.sh
  ![ -f "ComfyUI-Manager/node_db/dev/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/dev/scan.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-win.bat" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat

  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull


if OPTIONS['INSTALL_CUSTOM_NODES']:
  %cd $WORKSPACE
  %cd custom_nodes
  !git clone https://github.com/city96/ComfyUI-GGUF
  !pip install -r ComfyUI-GGUF/requirements.txt
  !git clone https://github.com/Fannovel16/comfyui_controlnet_aux
  !pip install -r comfyui_controlnet_aux/requirements.txt

%cd $WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !echo -= Install custom nodes dependencies =-
  !pip install GitPython
  !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies





/content/ComfyUI
-= Updating ComfyUI =-
Already up to date.
-= Install dependencies =-
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121, https://download.pytorch.org/whl/cu118, https://download.pytorch.org/whl/cu117
/content/ComfyUI/custom_nodes
/content/ComfyUI/custom_nodes/ComfyUI-Manager
Already up to date.
/content/ComfyUI
/content/ComfyUI/custom_nodes
fatal: destination path 'ComfyUI-GGUF' already exists and is not an empty directory.
fatal: destination path 'comfyui_controlnet_aux' already exists and is not an empty directory.
/content/ComfyUI
-= Install custom nodes dependencies =-

WARN: The `COMFYUI_PATH` environment variable is not set. Assuming 
`custom_nodes/ComfyUI-Manager/../../` as the ComfyUI path.
--------------------------------------------------------------------------------
--------------------
Restoring [1/3]: /content/ComfyUI/custom_nodes/ComfyUI-Manager
Install: pip packages

## ComfyUI-Manager: EXECUTE => ['/usr/bin/python3', '

### Download Resources

In [ ]:
# Download the FLUX model from Hugging Face and save it in the ./models/unet directory
!wget -c "https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q4_K_S.gguf" -P "./models/unet"

# Download the CLIP T5 model from Hugging Face and save it in ./models/clip
!wget -c "https://huggingface.co/city96/t5-v1_1-xxl-encoder-gguf/resolve/main/t5-v1_1-xxl-encoder-Q4_K_S.gguf" -P "./models/clip"

# Download the safetensors file for the CLIP model
!wget -c "https://huggingface.co/f5aiteam/CLIP/resolve/main/clip_l.safetensors" -P "./models/clip"

# Download the VAE model from Hugging Face and save it in ./models/vae
!wget -c "https://huggingface.co/f5aiteam/VAE/resolve/main/ae.safetensors" -P "./models/vae"

# ✅ Optional: Download VAE if needed
!wget -c "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/blob/main/vae-ft-mse-840000-ema-pruned.safetensors" -O "models/vae/vae-ft-mse-ema-pruned.safetensors"

# Download the LoRa model from Civitai.com Replace `xxxxxxxx` with your actual token
!wget -c "https://civitai.com/api/download/models/1198851?type=Model&format=SafeTensor&token=99e000c88fd944496f34cf7ae94fa791" -O "./models/loras/FluxThouS40k.safetensors"

# 📥 Download a base checkpoint model (e.g., Realistic Vision v5.1)
!wget -c "https://civitai.com/api/download/models/128713" -P "./models/checkpoints/realisticVisionV51_v51VAE.safetensors"

# Download the ControlNet models
# If you want to use a specific ControlNet model, remove the hashtag (#) from the corresponding line.
# !wget -c "https://huggingface.co/XLabs-AI/flux-controlnet-depth-v3/resolve/main/flux-depth-controlnet-v3.safetensors" -P "./models/controlnet"
# !wget -c "https://huggingface.co/XLabs-AI/flux-controlnet-collections/resolve/main/flux-canny-controlnet-v3.safetensors" -P "./models/controlnet"
# !wget -c "https://huggingface.co/XLabs-AI/flux-controlnet-collections/resolve/main/flux-hed-controlnet-v3.safetensors" -P "./models/controlnet"


--2025-07-03 16:23:01--  https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q4_K_S.gguf
Resolving huggingface.co (huggingface.co)... 18.164.174.55, 18.164.174.17, 18.164.174.118, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/3e/5c/3e5c1f82226fc1460150126b5513fa266f4b8aea7194d1a31c944ad9da31ae10/75bb19459b5240c9f373b5af527584af15b675867fa142efadf7478d6abbf62b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27flux1-dev-Q4_K_S.gguf%3B+filename%3D%22flux1-dev-Q4_K_S.gguf%22%3B&Expires=1751563381&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MTU2MzM4MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzNlLzVjLzNlNWMxZjgyMjI2ZmMxNDYwMTUwMTI2YjU1MTNmYTI2NmY0YjhhZWE3MTk0ZDFhMzFjOTQ0YWQ5ZGEzMWFlMTAvNzViYjE5NDU5YjUyNDBjOWYzNzNiNWFmNTI3NTg0YWYxNWI2NzU4NjdmYTE0MmVmYWRmNzQ3OGQ2YWJiZjYy

### Run ComfyUI

In [ ]:
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

--2025-07-03 16:29:51--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.6.1/cloudflared-linux-amd64.deb [following]
--2025-07-03 16:29:51--  https://github.com/cloudflare/cloudflared/releases/download/2025.6.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/f2a50f07-6f2f-47cd-8a16-5d3dca82800f?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250703%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250703T162951Z&X-Amz-Expires=1800&X-Amz-Signature=c3c28bd655d70572334fb66b19403b53833b56dd3b6dacaa257ad8cdea6